# [MMDetection](https://github.com/open-mmlab/mmdetection)
> Detect single neuronal cells in microscopy images

![](https://github.com/open-mmlab/mmdetection/raw/main/resources/mmdet-logo.png)

# 🛠 Install Libraries

In [1]:
!pip install -qqq /kaggle/input/mmdetv3-env/archive/addict-2.4.0-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/mmengine-0.7.4-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/mmcv-2.0.0-cp310-cp310-linux_x86_64.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/terminaltables-3.1.10-py2.py3-none-any.whl
# !pip install -qqq /kaggle/input/mmdetv3-env/archive/openmim-0.3.9-py2.py3-none-any.whl
!pip install -qqq /kaggle/input/pycocotools-206/wheels/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
# !pip install -qqq /kaggle/input/mmdet3-wheels-ando/mmdet-3.0.0-py3-none-any.whl
# !pip install -qqq /kaggle/input/weighted-boxes-fusion/ensemble_boxes-1.0.9-py3-none-any.whl

# !pip install -qqq /kaggle/input/mmdetection-3-1-evn/src/mmdet-3.1.0-py3-none-any.whl

In [2]:
!cp -r /kaggle/input/mmdetection-3-1-evn/src/mmdetection-main-3 /kaggle/working/
!mv /kaggle/working/mmdetection-main-3 /kaggle/working/mmdetection

In [3]:
%cd /kaggle/working/mmdetection
!pip install -e .
# %cd ..

/kaggle/working/mmdetection
Obtaining file:///kaggle/working/mmdetection
  Preparing metadata (setup.py) ... - \ done
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl size=101741 sha256=808a987dfdbc8a09bb35acd144dc2e394778d6ad1b485b39cc80c6fd95d472af
  Stored in directory: /root/.cache/pip/wheels/58/e6/f9/f87c8f8be098b51b616871315318329cae12cdb618f4caac93
Successfully built pycocotools
  Running setup.py develop for mmdet


In [4]:
# Check Pytorch installation
import torch, torchvision
print("torch version:",torch.__version__, "cuda:",torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print("mmdetection:",mmdet.__version__)

# Check mmcv installation
import mmcv
print("mmcv:",mmcv.__version__)

# Check mmengine installation
import mmengine
print("mmengine:",mmengine.__version__)

torch version: 2.0.0 cuda: True
mmdetection: 3.1.0
mmcv: 2.0.0
mmengine: 0.7.4


# 📚 Import Libraries

In [5]:
from itertools import groupby
from pycocotools import mask as mutils
from pycocotools.coco import COCO
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import wandb
from PIL import Image
import gc

from glob import glob
import matplotlib.pyplot as plt

from mmdet.utils import register_all_modules
from mmdet.apis import init_detector, inference_detector
from mmengine.visualization import Visualizer

import random
import json

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# ⚙️ Configuration

In [6]:
def set_random_seed(seed, deterministic=False):
    """Set random seed.

    Args:
        seed (int): Seed to be used.
        deterministic (bool): Whether to set the deterministic option for
            CUDNN backend, i.e., set `torch.backends.cudnn.deterministic`
            to True and `torch.backends.cudnn.benchmark` to False.
            Default: False.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [7]:
# set_random_seed(40)

In [8]:
!mkdir -p /kaggle/working/configs

In [9]:
%%writefile /kaggle/working/configs/custom_config.py

# model settings
model = dict(
    type='MaskRCNN',
    data_preprocessor=dict(
        type='DetDataPreprocessor',
        mean=[
            123.675,
            116.28,
            103.53,
        ],
        std=[
            58.395,
            57.12,
            57.375,
        ],
        bgr_to_rgb=True,
        pad_mask=True,
        pad_size_divisor=32),
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(
            0,
            1,
            2,
            3,
        ),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained',
                      checkpoint='torchvision://resnet101')),
    neck=dict(
        type='FPN',
        in_channels=[
            256,
            512,
            1024,
            2048,
        ],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[
                8,
            ],
            ratios=[
                0.5,
                1.0,
                2.0,
            ],
            strides=[
                4,
                8,
                16,
                32,
                64,
            ]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[
                0.0,
                0.0,
                0.0,
                0.0,
            ],
            target_stds=[
                1.0,
                1.0,
                1.0,
                1.0,
            ]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
    roi_head=dict(
        type='StandardRoIHead',
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[
                4,
                8,
                16,
                32,
            ]),
        bbox_head=dict(
            type='Shared2FCBBoxHead',
            in_channels=256,
            fc_out_channels=1024,
            roi_feat_size=7,
            num_classes=1,
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[
                    0.0,
                    0.0,
                    0.0,
                    0.0,
                ],
                target_stds=[
                    0.1,
                    0.1,
                    0.2,
                    0.2,
                ]),
            reg_class_agnostic=False,
            loss_cls=dict(
                type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
            loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
        mask_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=14, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[
                4,
                8,
                16,
                32,
            ]),
        mask_head=dict(
            type='FCNMaskHead',
            num_convs=4,
            in_channels=256,
            conv_out_channels=256,
            num_classes=1,
            loss_mask=dict(
                type='CrossEntropyLoss', use_mask=True, loss_weight=1.0))),
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=-1,
            pos_weight=-1,
            debug=False),
        rpn_proposal=dict(
            nms_across_levels=False,
            nms_pre=2000,
            nms_post=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.6,
                neg_iou_thr=0.6,
                min_pos_iou=0.6,
                match_low_quality=False,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=512,
                pos_fraction=0.25,
                neg_pos_ub=-1,
                add_gt_as_proposals=True),
            mask_size=28,
            pos_weight=-1,
            debug=False)),
    test_cfg=dict(
        rpn=dict(
            nms_across_levels=False,
            nms_pre=1000,
            nms_post=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            score_thr=0.05,
            nms=dict(type='nms', iou_thr=0.6),
            max_per_img=100,
            mask_thr_binary=0.5)))

dataset_type = 'CocoDataset'
data_root = ''
backend_args = None
metainfo = dict(
            classes=('blood_vessel', ),
            palette=[(255,0,0)]
            )

albu_train_transforms = [
    dict(
        type='ShiftScaleRotate',
        shift_limit=0.0625,
        scale_limit=0.15,
        rotate_limit=15,
        interpolation=1,
        p=0.2),
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.2),
    dict(type="Blur", p=1.0, blur_limit=7),
    dict(type='CLAHE', p=0.5),
    dict(type='Equalize', mode='cv', p=0.2),
    dict(
        type="OneOf",
        transforms=[
            dict(type="GaussianBlur", p=1.0, blur_limit=7),
            dict(type="MedianBlur", p=1.0, blur_limit=7),
        ],
        p=0.2,
    ),
    dict(type='RandomRotate90', p=0.2)

]

train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(
        type='LoadAnnotations', with_bbox=True, with_mask=True,
        poly2mask=True),
    dict(type='Resize', scale=(1300,800,), keep_ratio=True),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
            bbox_params=dict(
                type='BboxParams',
                format='pascal_voc',
                label_fields=['gt_bboxes_labels', 'gt_ignore_flags']),
                keymap=dict(img='image', gt_masks='masks', gt_bboxes='bboxes'),
            skip_img_without_anno=True),
    dict(type='RandomFlip', direction=['horizontal','vertical'], prob=0.6),
    dict(type='RandomCrop', crop_size=(640, 640)),
    dict(
    type='CachedMixUp',
        img_scale=(640, 640),
        ratio_range=(1.0, 1.0),
        max_cached_images=20,
        pad_val=(114, 114, 114)),
    dict(type='PackDetInputs'),
]
test_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='Resize', scale=(1300,800,), keep_ratio=True),
    dict(type='RandomFlip', direction=['horizontal','vertical',], prob=0.2),
    dict(
        type='PackDetInputs',
        meta_keys=(
            'img_id',
            'img_path',
            'ori_shape',
            'img_shape',
            'scale_factor',
        )),
]
train_dataloader = dict(
    batch_size=4,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type='CocoDataset',
        metainfo=dict(classes=('blood_vessel', ), palette=[(255,0,0,)]),
        data_root='',
        ann_file=
        '/kaggle/input/hubmap-coco-train-test-split/coco_annotations_train_split.json',
        data_prefix=dict(
            img='/kaggle/input/hubmap-hacking-the-human-vasculature/train/'),
        filter_cfg=dict(filter_empty_gt=True, min_size=32),
        pipeline=train_pipeline,
        backend_args=None))
val_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CocoDataset',
        metainfo=dict(classes=('blood_vessel', ), palette=[(255,0,0,)]),
        data_root='',
        ann_file=
        '/kaggle/input/hubmap-coco-train-test-split/coco_annotations_valid_split.json',
        data_prefix=dict(
            img='/kaggle/input/hubmap-hacking-the-human-vasculature/train/'),
        test_mode=True,
        pipeline=test_pipeline,
        backend_args=None))
test_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CocoDataset',
        metainfo=dict(classes=('blood_vessel', ), palette=[(255,0,0,)]),
        data_root='',
        ann_file=
        '/kaggle/input/hubmap-coco-train-test-split/coco_annotations_valid_split.json',
        data_prefix=dict(
            img='/kaggle/input/hubmap-hacking-the-human-vasculature/train/'),
        test_mode=True,
        pipeline=test_pipeline,
        backend_args=None))
val_evaluator = dict(
    type='CocoMetric',
    ann_file=
    '/kaggle/input/hubmap-coco-train-test-split/coco_annotations_valid_split.json',
    metric=[
        'segm',
    ],
    format_only=False,
    backend_args=None,
    iou_thrs=[
        0.6,
    ])
test_evaluator = dict(
    type='CocoMetric',
    ann_file=
    '/kaggle/input/hubmap-coco-train-test-split/coco_annotations_valid_split.json',
    metric=[
        'segm',
    ],
    format_only=False,
    backend_args=None,
    iou_thrs=[
        0.6,
    ])
optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='SGD', lr=0.02, momentum=0.9, weight_decay=0.0001),
    clip_grad=None)
param_scheduler = [
    dict(
        type='LinearLR', start_factor=0.001, by_epoch=False, begin=0, end=500),
    dict(
        type='MultiStepLR',
        by_epoch=True,
        begin=0,
        end=25,
        milestones=[
            11,
            13,
            15,
            17,
            19,
        ],
        gamma=0.1),
]
default_hooks = dict(
    timer=dict(type='IterTimerHook'),
    logger=dict(type='LoggerHook', interval=50),
    param_scheduler=dict(type='ParamSchedulerHook'),
    checkpoint=dict(
        type='CheckpointHook', interval=1, save_best='coco/segm_mAP'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    visualization=dict(type='DetVisualizationHook'))

custom_hooks = [
    dict(
        type='EarlyStoppingHook',
        monitor='coco/segm_mAP',
        rule='greater',
        min_delta=0.005,
        strict=False,
        check_finite=True,
        patience=6,
        stopping_threshold=None),
]

default_scope = 'mmdet'
env_cfg = dict(
    cudnn_benchmark=False,
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),
    dist_cfg=dict(backend='nccl'))
vis_backends = [
    dict(type='LocalVisBackend'),
]
visualizer = dict(
    type='DetLocalVisualizer',
    vis_backends=[
        dict(type='LocalVisBackend'),
    ],
    name='visualizer')
log_config = dict(hooks=[
    dict(type='TextLoggerHook'),
    dict(
        type='MMDetWandbHook',
        init_kwargs=dict(project='mmdetection'),
        interval=10,
        log_checkpoint=True,
        log_checkpoint_metadata=True,
        num_eval_images=100,
        bbox_score_thr=0.3),
])
log_processor = dict(type='LogProcessor', window_size=50, by_epoch=True)
log_level = 'INFO'
load_from = None
resume = False
train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=25, val_interval=1)
val_cfg = dict(type='ValLoop')
test_cfg = dict(type='TestLoop')
work_dir = '/content/work_dir'


Writing /kaggle/working/configs/custom_config.py


In [10]:
# coco1 = COCO('/kaggle/input/hubmap-coco-5-folds/coco_annotations_train_all_folds5_fold1.json')
# coco2 = COCO('/kaggle/input/hubmap-coco-pseudo/Pseudo-Labels.json')

# merged_anno = {
#     'images': coco1.dataset['images']+coco2.dataset['images'],
#     'annotations': coco1.dataset['annotations']+coco2.dataset['annotations'],
#     'categories': coco1.dataset['categories']+coco2.dataset['categories']
# }

# idx=1
# for i in range(len(merged_anno)):
#     merged_anno['annotations'][i]['id']=idx
    

# with open('/kaggle/working/train.json', 'w') as f:
#     json.dump(merged_anno, f)

In [11]:

# Đọc nội dung từ train_coco.json
with open('/kaggle/input/hubmap-coco-5-folds/coco_annotations_train_all_folds5_fold1.json', 'r') as file:
    coco_data = json.load(file)

# Đọc nội dung từ train_pseudo_coco.json
with open('/kaggle/input/hubmap-pseudo-dataset/Pseudo-Labels.json', 'r') as file:
    pseudo_coco_data = json.load(file)

# Nối dữ liệu
# coco_data['images'].extend(pseudo_coco_data['images'])
# coco_data['annotations'].extend(pseudo_coco_data['annotations'])

# # Ghi vào file mới
# with open('combined_coco.json', 'w') as file:
#     json.dump(coco_data, file)

In [12]:
offset = max(coco_data['annotations'], key=lambda x: x['id'])['id'] + 1

for annotation in pseudo_coco_data['annotations']:
    annotation['id'] += offset
#     annotation['image_id'] += offset

In [13]:
coco_data['annotations'].extend(pseudo_coco_data['annotations'])
coco_data['images'].extend(pseudo_coco_data['images'])

with open('/kaggle/working/combined_coco.json', 'w') as file:
    json.dump(coco_data, file)

In [14]:
%%writefile /kaggle/working/configs/custom_config.py

# model settings

# _base_ = '/kaggle/working/mmdetection/configs/mask_rcnn/mask-rcnn_x101-32x4d_fpn_1x_coco.py'

model = dict(
    type='MaskRCNN',
    data_preprocessor=dict(
        type='DetDataPreprocessor',
        mean=[
            123.675,
            116.28,
            103.53,
        ],
        std=[
            58.395,
            57.12,
            57.375,
        ],
        bgr_to_rgb=True,
        pad_mask=True,
        pad_size_divisor=32),
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(
            0,
            1,
            2,
            3,
        ),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained',
                      checkpoint='torchvision://resnet101')),
    neck=dict(
        type='FPN',
        in_channels=[
            256,
            512,
            1024,
            2048,
        ],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[
                8,
            ],
            ratios=[
                0.5,
                1.0,
                2.0,
            ],
            strides=[
                4,
                8,
                16,
                32,
                64,
            ]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[
                0.0,
                0.0,
                0.0,
                0.0,
            ],
            target_stds=[
                1.0,
                1.0,
                1.0,
                1.0,
            ]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
    roi_head=dict(
        type='StandardRoIHead',
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[
                4,
                8,
                16,
                32,
            ]),
        bbox_head=dict(
            type='Shared2FCBBoxHead',
            in_channels=256,
            fc_out_channels=1024,
            roi_feat_size=7,
            num_classes=1,
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[
                    0.0,
                    0.0,
                    0.0,
                    0.0,
                ],
                target_stds=[
                    0.1,
                    0.1,
                    0.2,
                    0.2,
                ]),
            reg_class_agnostic=False,
            loss_cls=dict(
                type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
            loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
        mask_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=14, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[
                4,
                8,
                16,
                32,
            ]),
        mask_head=dict(
            type='FCNMaskHead',
            num_convs=4,
            in_channels=256,
            conv_out_channels=256,
            num_classes=1,
            loss_mask=dict(
                type='CrossEntropyLoss', use_mask=True, loss_weight=1.0))),
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=-1,
            pos_weight=-1,
            debug=False),
        rpn_proposal=dict(
            nms_across_levels=False,
            nms_pre=2000,
            nms_post=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.6,
                neg_iou_thr=0.6,
                min_pos_iou=0.6,
                match_low_quality=False,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=512,
                pos_fraction=0.25,
                neg_pos_ub=-1,
                add_gt_as_proposals=True),
            mask_size=28,
            pos_weight=-1,
            debug=False)),
    test_cfg=dict(
        rpn=dict(
            nms_across_levels=False,
            nms_pre=1000,
            nms_post=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            score_thr=0.05,
            nms=dict(type='nms', iou_thr=0.6),
            max_per_img=100,
            mask_thr_binary=0.5)))
dataset_type = 'CocoDataset'
data_root = ''
backend_args = None
metainfo = dict(
    classes=('blood_vessel', ), palette=[
        (
            255,
            0,
            0,
        ),
    ])

img_size = (1024, 1024)
albu_train_transforms = [
    dict(type='RandomRotate90', p=0.2),
    dict(type='ChannelShuffle', p=0.1)
]


train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(
        type='LoadAnnotations', with_bbox=True, with_mask=True,
        poly2mask=True),
    dict(type='Resize', scale=img_size, keep_ratio=True),
    dict(
        type='CachedMosaic',
        img_scale=(780, 780),
        pad_val=0.0,
        max_cached_images=4,
        random_pop=False),
    dict(type='Albu', transforms = albu_train_transforms,
         bbox_params=dict(type="BboxParams",
                          format='pascal_voc',
                          label_fields=['gt_bboxes_labels', 'gt_ignore_flags'],
                          min_visibility=0.0,
                          filter_lost_elements=True
                         ),
         keymap=dict(img='image', gt_masks='masks', gt_bboxes='bboxes'),
         skip_img_without_anno=True
        ),
    dict(type='RandomFlip', direction=['horizontal','vertical'], prob=0.6),
    dict(type='PackDetInputs'),
]

test_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='Resize', scale=img_size, keep_ratio=True),
    dict(
        type='PackDetInputs',
        meta_keys=(
            'img_id',
            'img_path',
            'ori_shape',
            'img_shape',
            'scale_factor',
        )),
]
train_dataloader = dict(
    batch_size=4,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type='CocoDataset',
        metainfo=dict(classes=('blood_vessel', ), palette=[
            (
                255,
                0,
                0,
            ),
        ]),
        data_root='',
        ann_file=
        '/kaggle/working/combined_coco.json',
        data_prefix=dict(
            img='/kaggle/input/hubmap-hacking-the-human-vasculature/train/'),
        filter_cfg=dict(filter_empty_gt=True, min_size=32),
        pipeline=train_pipeline,
        backend_args=None))
val_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CocoDataset',
        metainfo=dict(classes=('blood_vessel', ), palette=[
            (
                255,
                0,
                0,
            ),
        ]),
        data_root='',
        ann_file=
        '/kaggle/input/hubmap-coco-5-folds/coco_annotations_valid_all_folds5_fold1.json',
        data_prefix=dict(
            img='/kaggle/input/hubmap-hacking-the-human-vasculature/train/'),
        test_mode=True,
        pipeline=[
            dict(type='LoadImageFromFile', backend_args=None),
            dict(type='Resize', scale=img_size, keep_ratio=True),
            dict(
                type='PackDetInputs',
                meta_keys=(
                    'img_id',
                    'img_path',
                    'ori_shape',
                    'img_shape',
                    'scale_factor',
                )),
        ],
        backend_args=None))
test_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CocoDataset',
        metainfo=dict(classes=('blood_vessel', ), palette=[
            (
                255,
                0,
                0,
            ),
        ]),
        data_root='',
        ann_file=
        '/kaggle/input/hubmap-coco-5-folds/coco_annotations_valid_all_folds5_fold1.json',
        data_prefix=dict(
            img='/kaggle/input/hubmap-hacking-the-human-vasculature/train/'),
        test_mode=True,
        pipeline=[
            dict(type='LoadImageFromFile', backend_args=None),
            dict(type='Resize', scale=img_size, keep_ratio=True),
            dict(
                type='PackDetInputs',
                meta_keys=(
                    'img_id',
                    'img_path',
                    'ori_shape',
                    'img_shape',
                    'scale_factor',
                )),
        ],
        backend_args=None))
val_evaluator = dict(
    type='CocoMetric',
    ann_file=
    '/kaggle/input/hubmap-coco-5-folds/coco_annotations_valid_all_folds5_fold1.json',
    metric=[
        'segm',
    ],
    format_only=False,
    backend_args=None,
    iou_thrs=[
        0.6,
    ])
test_evaluator = dict(
    type='CocoMetric',
    ann_file=
    '/kaggle/input/hubmap-coco-5-folds/coco_annotations_valid_all_folds5_fold1.json',
    metric=[
        'segm',
    ],
    format_only=False,
    backend_args=None,
    iou_thrs=[
        0.6,
    ])
optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='SGD', lr=0.02, momentum=0.9, weight_decay=0.0001),
    clip_grad=None)
param_scheduler = [
    dict(
        type='LinearLR', start_factor=0.001, by_epoch=False, begin=0, end=500),
    dict(
        type='MultiStepLR',
        by_epoch=True,
        begin=0,
        end=25,
        milestones=[
            13,
            16,
            19,
            22
        ],
        gamma=0.1),
]
default_hooks = dict(
    timer=dict(type='IterTimerHook'),
    logger=dict(type='LoggerHook', interval=50),
    param_scheduler=dict(type='ParamSchedulerHook'),
    checkpoint=dict(
        type='CheckpointHook', interval=1, save_best='coco/segm_mAP'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    visualization=dict(type='DetVisualizationHook'))
custom_hooks = [
    dict(
        type='EarlyStoppingHook',
        monitor='coco/segm_mAP',
        rule='greater',
        min_delta=0.005,
        strict=False,
        check_finite=True,
        patience=10,
        stopping_threshold=None),
]
default_scope = 'mmdet'
env_cfg = dict(
    cudnn_benchmark=False,
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),
    dist_cfg=dict(backend='nccl'))
vis_backends = [
    dict(type='LocalVisBackend'),
]
visualizer = dict(
    type='DetLocalVisualizer',
    vis_backends=[
        dict(type='LocalVisBackend'),
    ],
    name='visualizer')
log_config = dict(hooks=[
    dict(type='TextLoggerHook'),
    dict(
        type='MMDetWandbHook',
        init_kwargs=dict(project='mmdetection'),
        interval=10,
        log_checkpoint=True,
        log_checkpoint_metadata=True,
        num_eval_images=100,
        bbox_score_thr=0.3),
])
log_processor = dict(type='LogProcessor', window_size=50, by_epoch=True)
log_level = 'INFO'
load_from = None
resume = False
train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=25, val_interval=1)
val_cfg = dict(type='ValLoop')
test_cfg = dict(type='TestLoop')

Overwriting /kaggle/working/configs/custom_config.py


In [15]:
%mkdir /kaggle/working/work_dir

In [16]:
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules

In [17]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value = user_secrets.get_secret("wandb")

# import wandb
# wandb.login(key=secret_value)

In [18]:
cfg = Config.fromfile("/kaggle/working/configs/custom_config.py")
cfg.work_dir = f"/kaggle/working/work_dir"

vis_backends = [dict(type='LocalVisBackend')]
cfg.visualizer = dict(type='DetLocalVisualizer',
                      vis_backends=vis_backends,
                      name='visualizer')

runner = Runner.from_cfg(cfg)


07/20 11:00:41 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
    CUDA available: True
    numpy_random_seed: 837273438
    GPU 0: Tesla P100-PCIE-16GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: gcc (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0
    PyTorch: 2.0.0
    PyTorch compiling details: PyTorch built with:
  - GCC 11.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gen

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

07/20 11:01:00 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
07/20 11:01:00 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
(LOWEST      ) EarlyStoppingHook                  
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_i

In [19]:
runner.train()

loading annotations into memory...
Done (t=0.85s)
creating index...
index created!
loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
07/20 11:01:06 - mmengine - INFO - load model from: torchvision://resnet101
07/20 11:01:06 - mmengine - INFO - Loads checkpoint by torchvision backend from path: torchvision://resnet101


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


07/20 11:01:09 - mmengine - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

07/20 11:01:09 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
07/20 11:01:10 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
07/20 11:01:10 - mmengine - INFO - Checkpoints will be saved to /kaggle/working/work_dir.
07/20 11:02:05 - mmengine - INFO - Epoch(train)  [1][  50/1180]  lr: 1.9820e-03  eta: 9:09:02  time: 1.1186  data_time: 0.0810  memory: 10294  loss: 1.6997  loss_rpn_cls: 0.5919  loss_rpn_bbox: 0.0473  loss_cls: 0.1778  acc: 99.1699  loss_bbox: 0.0059  loss_mask: 0.8769
07/20 11:02:57 - mmengine - INFO - Epoch(train)  [1][ 100/1180]  lr: 3.9840e-03  eta: 8:45:27  time: 1.0261  data_time: 0.0642  memory: 10296  loss: 0.8680  loss_rpn_cls: 0.2338  loss_

MaskRCNN(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)

# 🔎 Prediction

In [20]:
# path_list = ["/kaggle/input/hubmap-hacking-the-human-vasculature/test/"+i for i in os.listdir("/kaggle/input/hubmap-hacking-the-human-vasculature/test/")]
# path_list

# img = mmcv.imread("/kaggle/input/hubmap-hacking-the-human-vasculature/test/72e40acccadf.tif")
# checkpoint_file = "/kaggle/working/work_dir/best_coco_segm_mAP_epoch_4.pth"

# model = init_detector(cfg, checkpoint=checkpoint_file, device="cuda:0")
# new_result = inference_detector(model, imgs=path_list)
# # print(new_result)

# visualizer_now = Visualizer.get_current_instance()

# visualizer_now.dataset_meta = model.dataset_meta
# visualizer_now.set_image(img)
# visualizer_now.add_datasample(
#     'new_result',
#     img,
#     data_sample=new_result[0],
#     draw_gt=True,
#     draw_pred=True,
#     wait_time=0,
# )
# visualizer_now.show()